# Breast Cancer Histology Imaging
mkfold.py -- from: https://web.inf.ufpr.br/vri/databases/breast-cancer-histopathological-database-breakhis/
(Spanhol et al., 2016)

In [ ]:
# Run only one time!

# %run mkfold.py

# Keras Modeling
Processing and SimpleCNN models adapted from: https://www.analyticsvidhya.com/blog/2020/10/create-image-classification-model-python-keras/

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils.multiclass import unique_labels
import matplotlib.image as mpimg
import itertools

import cv2
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizer_v1 import Adam
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Labeling data with 2 classes

# labels = ['B', 'M']
img_size = 224
def get_data(data_dir):
    data = [] 
    path = data_dir
    for img in os.listdir(path):
        if img[4] == 'B':
            class_num = 0
        else:
            class_num = 1
        try:
            img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
            resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
            data.append([resized_arr, class_num])
        except Exception as e:
            print(e)
    return np.array(data)

# Import data from all magnifications

In [ ]:
# 40X MAG

train41 = get_data('./fold1/train/40X')
val41 = get_data('./fold1/test/40X')

train42 = get_data('./fold2/train/40X')
val42 = get_data('./fold2/test/40X')

train43 = get_data('./fold3/train/40X')
val43 = get_data('./fold3/test/40X')

train44 = get_data('./fold4/train/40X')
val44 = get_data('./fold4/test/40X')

train45 = get_data('./fold5/train/40X')
val45= get_data('./fold5/test/40X')

train_set40 = [train41, train42, train43, train44, train45]
val_set40 = [val41, val42, val43, val44, val45]

In [ ]:
# 100 MAG

train1 = get_data('./fold1/train/100X')
val1 = get_data('./fold1/test/100X')

train2 = get_data('./fold2/train/100X')
val2 = get_data('./fold2/test/100X')

train3 = get_data('./fold3/train/100X')
val3 = get_data('./fold3/test/100X')

train4 = get_data('./fold4/train/100X')
val4 = get_data('./fold4/test/100X')

train5 = get_data('./fold5/train/100X')
val5= get_data('./fold5/test/100X')

train_set100 = [train1, train2, train3, train4, train5]
val_set100 = [val1, val2, val3, val4, val5]

In [ ]:
# 200X Mag

train21 = get_data('./fold1/train/200X')
val21 = get_data('./fold1/test/200X')

train22 = get_data('./fold2/train/200X')
val22 = get_data('./fold2/test/200X')

train23 = get_data('./fold3/train/200X')
val23 = get_data('./fold3/test/200X')

train24 = get_data('./fold4/train/200X')
val24 = get_data('./fold4/test/200X')

train25 = get_data('./fold5/train/200X')
val25= get_data('./fold5/test/200X')

train_set200 = [train21, train22, train23, train24, train25]
val_set200 = [val21, val22, val23, val24, val25]

In [ ]:
# 400X Mag
train421 = get_data('./fold1/train/400X')
val421 = get_data('./fold1/test/400X')

train422 = get_data('./fold2/train/400X')
val422 = get_data('./fold2/test/400X')

train423 = get_data('./fold3/train/400X')
val423 = get_data('./fold3/test/400X')

train424 = get_data('./fold4/train/400X')
val424 = get_data('./fold4/test/400X')

train425 = get_data('./fold5/train/400X')
val425= get_data('./fold5/test/400X')

train_set400 = [train421, train422, train423, train424, train425]
val_set400 = [val421, val422, val423, val424, val425]

In [ ]:
# Data Preprocessing and Augmentation

def processing(train, val):
    x_train = []
    y_train = []
    x_val = []
    y_val = []

    for feature, label in train:
        x_train.append(feature)
        y_train.append(label)

    for feature, label in val:
        x_val.append(feature)
        y_val.append(label)

    # Normalize the data
    x_train = np.array(x_train) /255
    x_val = np.array(x_val) /255

    x_train.reshape(-1, img_size, img_size, 1)
    y_train = np.array(y_train)

    x_val.reshape(-1, img_size, img_size, 1)
    y_val = np.array(y_val)
    
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    datagen.fit(x_train)
    
    return x_train, y_train, x_val, y_val

# SimpleCNN and SimpleCNN2 Models

In [ ]:
# Define simple CNN model with 3 Convolutional Layers followed by max-pooling layers + dropout layer (to avoid overfitting)
def simpleCNN():
    model = Sequential()
    model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(224,224,3)))
    model.add(MaxPool2D())

    model.add(Conv2D(32, 3, padding="same", activation="relu"))
    model.add(MaxPool2D())

    model.add(Conv2D(64, 3, padding="same", activation="relu"))
    model.add(MaxPool2D())
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128,activation="relu"))
    model.add(Dense(2, activation="softmax"))

    model.summary()
    
    return model

In [ ]:
# Define simple CNN model with 3 Convolutional Layers followed by max-pooling layers + dropout layer (to avoid overfitting)
def simpleCNN_2():
    model = Sequential()
    model.add(Conv2D(64, 3, padding="same", activation="relu", input_shape=(224,224,3)))
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, 3,padding="same", activation="relu"))
    model.add(MaxPool2D())

    model.add(Conv2D(256, 3, padding="same", activation="relu"))
    model.add(MaxPool2D())
    
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(128,activation="relu"))
    model.add(Dense(2, activation="softmax"))

    model.summary()
    
    return model

In [ ]:
def run_CNN(x_train, y_train, x_val, y_val, model, epochs):
    # Using Adam Optimzer and SparseCategoricalCrossentropy 
    opt = tf.keras.optimizers.Adam(learning_rate=0.000001)
    ls = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer = opt , loss = ls , metrics = ['accuracy'])
    
    history = model.fit(x_train, y_train, epochs=epochs, validation_data = (x_val, y_val))
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(epochs)

    plt.figure(figsize=(15, 15))
    plt.subplot(2, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()
    
    predict_x=model.predict(x_val) 
    predictions=np.argmax(predict_x,axis=1)
    predictions = predictions.reshape(1,-1)[0]

    print(classification_report(y_val, predictions, target_names = ['Benign (Class 0)','Malignant (Class 1)']))
    print(confusion_matrix(y_val, predictions))

# Run SimpleCNN on all Magnifications

In [ ]:
# SimpleCNN model summary
model = simpleCNN()

In [ ]:
# 40X

for i in range(len(train_set40)):
    print("THIS IS FOLD", i+1)
    x_train, y_train, x_val, y_val = processing(train_set40[i], val_set40[i])
    model = simpleCNN()
    run_CNN(x_train, y_train, x_val, y_val, model, 100) 

In [ ]:
# 100X

for i in range(len(train_set40)):
    print("THIS IS FOLD", i+1)
    x_train, y_train, x_val, y_val = processing(train_set100[i], val_set100[i])
    model = simpleCNN()
    run_CNN(x_train, y_train, x_val, y_val, model, 100)  

In [ ]:
# 200X

for i in range(len(train_set200)):
    print("THIS IS FOLD", i+1)
    x_train, y_train, x_val, y_val = processing(train_set200[i], val_set200[i])
    model = simpleCNN()
    run_CNN(x_train, y_train, x_val, y_val, model, 100)  

In [ ]:
# 400X

for i in range(len(train_set400)):
    print("THIS IS FOLD", i+1)
    x_train, y_train, x_val, y_val = processing(train_set400[i], val_set400[i])
    model = simpleCNN()
    run_CNN(x_train, y_train, x_val, y_val, model, 100)  

# SimpleCNN2

In [ ]:
# SimpleCNN2 Model Summary
model = simpleCNN_2()

In [ ]:
# SimpleCNN2, 200X Mag (all folds)

for i in range(len(train_set200)):
    print("THIS IS FOLD", i+1)
    x_train, y_train, x_val, y_val = processing(train_set200[i], val_set200[i])
    model = simpleCNN_2()
    run_CNN(x_train, y_train, x_val, y_val, model, 100)  

In [ ]:
trainnew200 = [train23, train24, train25]
valnew200 = [val23, val24, val25]

In [ ]:
# SimpleCNN2, 200X Mag (folds 3-5 after force quit)

for i in range(len(trainnew200)):
    print("THIS IS FOLD", i+3)
    x_train, y_train, x_val, y_val = processing(trainnew200[i], valnew200[i])
    model = simpleCNN()
    run_CNN(x_train, y_train, x_val, y_val, model, 100)  

# DenseNet
Model adapted from https://www.pluralsight.com/guides/introduction-to-densenet-with-tensorflow

In [ ]:
def DenseNetModel(x_train, y_train, x_val, y_val, epochs):
    model_d=DenseNet121(weights='imagenet',include_top=False, input_shape=(224, 224, 3)) 

    x=model_d.output

    x= GlobalAveragePooling2D()(x)
    x= BatchNormalization()(x)
    x= Dropout(0.5)(x)
    x= Dense(1024,activation='relu')(x) 
    x= Dense(512,activation='relu')(x) 
    x= BatchNormalization()(x)
    x= Dropout(0.5)(x)

    preds=Dense(2,activation='softmax')(x) #FC-layer
    
    model=Model(model_d.input, preds)
    
    for layer in model.layers[:-8]:
        layer.trainable=False
    
    for layer in model.layers[-8:]:
        layer.trainable=True
    
#     model.summary()
    
    model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
    
    anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
    checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)
    
    # Fits-the-model
    history = model.fit(x_train, y_train,
                   epochs=epochs,
                   verbose=1,
                   callbacks=[anne, checkpoint],
                   validation_data=(x_val, y_val))
    
    return history

In [ ]:
def runDenseNet(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(epochs)

    plt.figure(figsize=(15, 15))
    plt.subplot(2, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()
    
    predict_x=model.predict(x_val) 
    predictions=np.argmax(predict_x,axis=1)
    predictions = predictions.reshape(1,-1)[0]
    
    rounded_labels=np.argmax(y_val, axis=1)

    print(classification_report(rounded_labels, predictions))
    
    cm = confusion_matrix(rounded_labels, predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.show()

In [ ]:
epochs = 50
for i in range(len(train_set200)):
    print("THIS IS FOLD", i+1)
    x_train, y_train, x_val, y_val = processing(train_set200[i], val_set200[i])
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=2)
    history = DenseNetModel(x_train, y_train, x_val, y_val, epochs) 
    runDenseNet(history)

# VGG16
Model adapted from https://towardsdatascience.com/step-by-step-vgg16-implementation-in-keras-for-beginners-a833c686ae6c and https://medium.com/analytics-vidhya/multi-class-image-classification-using-transfer-learning-with-deep-convolutional-neural-networks-eab051cde3fb

In [ ]:
# Attempt VGG16 from scratch-- did not work well
# Adapted from https://towardsdatascience.com/step-by-step-vgg16-implementation-in-keras-for-beginners-a833c686ae6c 

# def VGG16_1(x_train, y_train, x_val, y_val, vgg_epoch):
#     model = Sequential()
#     model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
#     model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
#     model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    
#     model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
#     model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
#     model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    
#     model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
#     model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
#     model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
#     model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    
#     model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
#     model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
#     model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
#     model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    
#     model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
#     model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
#     model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
#     model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    
#     model.add(Flatten())
#     model.add(Dense(units=4096,activation="relu"))
#     model.add(Dense(units=4096,activation="relu"))
#     model.add(Dense(units=2, activation="softmax"))
    
#     model.summary()
    
#     y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
#     y_val = tf.keras.utils.to_categorical(y_val, num_classes=2)
    
#     number_of_epochs = vgg_epoch
#     vgg16_filepath = '/tmp/checkpoint'
#     vgg_checkpoint = tf.keras.callbacks.ModelCheckpoint(vgg16_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#     vgg_early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15)
    
    
#     opt = tf.keras.optimizers.Adam(learning_rate=0.001)
#     ls = tf.keras.losses.CategoricalCrossentropy()
#     opt = Adam(lr=0.001)
#     model.compile(optimizer=opt, loss=ls, metrics=['accuracy'])
#     history = model.fit(x_train, y_train, epochs=epochs, validation_data = (x_val, y_val), callbacks=[vgg_checkpoint,vgg_early_stopping],verbose=1)
    
#     return model, history

In [ ]:
# Transfer-Learning: Pretrained VGG16
# Adapted from https://medium.com/analytics-vidhya/multi-class-image-classification-using-transfer-learning-with-deep-convolutional-neural-networks-eab051cde3fb

def VGG16_2(x_train, y_train, x_val, y_val, vgg_epoch):
    vgg16_model = VGG16(pooling='avg', weights='imagenet', include_top=False, input_shape=(224,224,3))
    for layers in vgg16_model.layers:
        layers.trainable=False
    last_output = vgg16_model.layers[-1].output
    vgg_x = Flatten()(last_output)
    vgg_x = Dense(128, activation = 'relu')(vgg_x)
    vgg_x = Dense(128, activation = 'relu')(vgg_x)
    vgg_x = Dense(2, activation = 'softmax')(vgg_x)
    vgg16_final_model = tf.keras.Model(vgg16_model.input, vgg_x)
    vgg16_final_model.compile(loss = 'categorical_crossentropy', optimizer= 'adam', metrics=['acc'])
    
    vgg16_final_model.summary()

#   VGG16
    number_of_epochs = vgg_epoch
    vgg16_filepath = 'vgg_16_'+'-saved-model-{epoch:02d}-acc-{val_acc:.2f}.hdf5'
    vgg_checkpoint = tf.keras.callbacks.ModelCheckpoint(vgg16_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    vgg_early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=5)
    
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=2)
    
    vgg16_history = vgg16_final_model.fit(x_train, y_train, epochs = number_of_epochs, validation_data = (x_val, y_val),callbacks=[vgg_checkpoint,vgg_early_stopping],verbose=1)

    return vgg16_final_model 

In [ ]:
def runVGG16(history):
    # Using Adam Optimzer and SparseCategoricalCrossentropy     
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(21)

    plt.figure(figsize=(15, 15))
    plt.subplot(2, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()
    
    predict_x=model.predict(x_val) 
    predictions=np.argmax(predict_x,axis=1)
    predictions = predictions.reshape(1,-1)[0]

    print(classification_report(y_val, predictions, target_names = ['Benign (Class 0)','Malignant (Class 1)']))
    print(confusion_matrix(y_val, predictions))

In [ ]:
def VGG_data(model_vgg16, epochs):
    acc = model_vgg16.history.history['acc']
    val_acc = model_vgg16.history.history['val_acc']
    loss = model_vgg16.history.history['loss']
    val_loss = model_vgg16.history.history['val_loss']
    epochs_range = range(epochs)
    
    print(val_acc)

    predict_x= model_vgg16.predict(x_val) 
    predictions=np.argmax(predict_x,axis=1)
    predictions = predictions.reshape(1,-1)[0]

    print(classification_report(y_val, predictions, target_names = ['Benign (Class 0)','Malignant (Class 1)']))
    cm = confusion_matrix(y_val, predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.show()    
    
    plt.figure(figsize=(15, 15))
    plt.subplot(2, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

In [ ]:
## Pretrained VGG16 Model Summary
VGG16_2()

In [ ]:
model_vgg16.summary()

## Run VGG16_2 on 200x

In [ ]:
# 200X
epochs = 50

for i in range(len(train_set200)):
    print("THIS IS FOLD", i+1)
    x_train, y_train, x_val, y_val = processing(train_set200[i], val_set200[i])
    model_vgg16 = VGG16_2(x_train, y_train, x_val, y_val, epochs)
    VGG_data(model_vgg16, epochs)    